In [2]:
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'Utilizez: {device}')


Utilizez: cuda


In [3]:
import torch


if torch.cuda.is_available():

    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [4]:
import pandas as pd
df_fake= pd.read_csv("../data/Fake.csv")
df_true= pd.read_csv("../data/True.csv")



df_fake["label"] = 0
df_true["label"] = 1



In [5]:
import re
def remove_city_newsletter(text):
    return re.sub(r"^[A-Za-z/,\s]+(?:\([A-Za-z]+\))?\s-\s", '', text)

def remove_html_caracters(text):
    html_special_chars_pattern = r'&[a-zA-Z0-9#]+;|<|>'
    return re.sub(html_special_chars_pattern,'',text)

def remove_links(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_hastags(text):
    return re.sub(r'#\S+', '', text)

def remove_image(text):
     return re.sub(r'\b(Featured|Photo\sby|Image)\w*\b.*', '', text)

def remove_coments(text):
    regex_date_username = r"((january|february|march|april|may|june|july|august|september|october|november|december)\s\d{1,2},\s\d{4}).*?@(?!realDonaldTrump)\w+"
    return re.sub(regex_date_username,'', text, flags=re.IGNORECASE) #eliminarea datelor
def lower_text(text):
    return text.lower()

df_fake['text'] = df_fake['text'].apply(remove_hastags)
df_fake['text'] = df_fake['text'].apply(remove_html_caracters)
df_fake['text'] = df_fake['text'].apply(remove_links)
df_fake['text'] = df_fake['text'].apply(remove_image)
df_fake['text'] = df_fake['text'].apply(remove_coments)
df_fake['text'] = df_fake['text'].apply(lower_text)


df_true['text'] = df_true['text'].apply(remove_city_newsletter)
df_true['text'] = df_true['text'].apply(remove_hastags)
df_true['text'] = df_true['text'].apply(remove_html_caracters)
df_true['text'] = df_true['text'].apply(remove_links)
df_true['text'] = df_true['text'].apply(lower_text)


In [6]:
import pandas as pd
from datasets import DatasetDict, Dataset

frames=[df_true,df_fake]
df=pd.concat(frames)
shuffled_df = df.sample(frac=1).reset_index(drop=True)
shuffled_df = shuffled_df.drop_duplicates()
shuffled_selected = shuffled_df[['text', 'label']]

news_text=shuffled_selected
train_frac = 0.7
valid_frac = 0.15
test_frac = 0.15
#
train_size = int(train_frac * len(news_text))
valid_size = int(valid_frac * len(news_text))
#
# create train, validation, and test datasets
train_df = news_text[:train_size]
valid_df = news_text[train_size:train_size + valid_size]
test_df = news_text[train_size + valid_size:]
#
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)
dataset_dict = DatasetDict({
    'train': train_ds,
    'validation': valid_ds,
    'test': test_ds
})


print(dataset_dict)

D:\github\AI-examples\AI_examples\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'subject', 'date', 'label', '__index_level_0__'],
        num_rows: 31282
    })
    validation: Dataset({
        features: ['title', 'text', 'subject', 'date', 'label', '__index_level_0__'],
        num_rows: 6703
    })
    test: Dataset({
        features: ['title', 'text', 'subject', 'date', 'label', '__index_level_0__'],
        num_rows: 6704
    })
})


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_path="google-bert/bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_path)
id2label={0:"fake",1:"true"}
label2id={"fake":0,"true":1}
model=AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2,id2label=id2label,label2id=label2id)

D:\github\AI-examples\AI_examples\.venv\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
for name,param in model.base_model.named_parameters():
    param.requires_grad=False
for name,param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad=True

In [9]:
from transformers import DataCollatorWithPadding
def preprocess_function(examples):
    return tokenizer(
        examples["text"], 
        truncation=True,  
        padding="max_length",  
        max_length=512  
    )

tokenized_data=dataset_dict.map(preprocess_function,batched=True)
tokenized_data = tokenized_data.remove_columns(["__index_level_0__", "date", "text", "title", "subject"])
data_colator= DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 6704/6704 [00:03<00:00, 2204.82 examples/s]


In [10]:
import evaluate
import numpy as np
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)

    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]

    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'], 3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)

    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'], 3)

    return {"Accuracy": acc, "AUC": auc}

In [11]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="bert_training_final",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=400,
    logging_dir="./logs",
    report_to="none" 
)

In [12]:
import torch
print(torch.cuda.get_device_name(0))  # If no GPU, this will raise an error


NVIDIA GeForce RTX 3050 Laptop GPU


In [13]:
from transformers import Trainer
from transformers import EarlyStoppingCallback
import torch

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2
)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_colator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
    
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.109500,0.094076,0.971000,0.994000
2,0.094600,0.072883,0.979000,0.996000
3,0.067800,0.074067,0.977000,0.997000
4,0.077400,0.066433,0.981000,0.997000
5,0.074800,0.067674,0.979000,0.997000


TrainOutput(global_step=19555, training_loss=0.09732038146301597, metrics={'train_runtime': 7534.907, 'train_samples_per_second': 20.758, 'train_steps_per_second': 2.595, 'total_flos': 4.11532001688576e+16, 'train_loss': 0.09732038146301597, 'epoch': 5.0})

In [17]:
predictions=trainer.predict(tokenized_data["validation"])

logits=predictions.predictions
labels=predictions.label_ids
metrics=compute_metrics((logits,labels))
print(metrics)

{'Accuracy': np.float64(0.981), 'AUC': np.float64(0.998)}


In [15]:
model.save_pretrained("./bert_fake_news_model_final")
tokenizer.save_pretrained("./bert_fake_news_model_final")


('./bert_fake_news_model_final\\tokenizer_config.json',
 './bert_fake_news_model_final\\special_tokens_map.json',
 './bert_fake_news_model_final\\vocab.txt',
 './bert_fake_news_model_final\\added_tokens.json',
 './bert_fake_news_model_final\\tokenizer.json')

In [16]:
dataset_dict.save_to_disk("./my_news_dataset_final")

Saving the dataset (1/1 shards): 100%|██████████| 6704/6704 [00:00<00:00, 162235.25 examples/s]
